In [16]:
!pip install kagglehub[pandas-datasets]


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
# %load_ext autoreload
# %autoreload 2
# %reload_ext autoreload

import os
ROOT_DIR = '/workspace/NN'
os.chdir(ROOT_DIR)

import shutil
import kagglehub
import torch

dataset_path = os.path.join(ROOT_DIR, 'neural', 'datasets', 'fashionmnist')
logs_path = os.path.join(ROOT_DIR, 'neural', 'logs')
csv_file = os.path.join(dataset_path, 'fashion-mnist_train.csv')
prod_weights_path = os.path.join(ROOT_DIR, 'neural', 'weights', 'prod')
test_weights_path = os.path.join(ROOT_DIR, 'neural', 'weights', 'lab_1')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
from pytorch_lightning.callbacks import ModelCheckpoint
from torchmetrics.classification import MulticlassF1Score
import uuid
from datetime import datetime
import time
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import FashionMNIST

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

import torchmetrics

# Определяем лёгкую сверточную сеть для FashionMNIST
class LightweightFashionMNIST(nn.Module):
    def __init__(self):
        super(LightweightFashionMNIST, self).__init__()
        # Входное изображение: 1 x 28 x 28
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)   # -> 16 x 28 x 28
        self.pool  = nn.MaxPool2d(2, 2)                             # -> 16 x 14 x 14
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)    # -> 32 x 14 x 14
        # После второго pooling: 32 x 7 x 7
        self.fc1   = nn.Linear(32 * 7 * 7, 64)
        self.fc2   = nn.Linear(64, 10)  # 10 классов

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Определяем LightningModule
class FashionMNISTLitModel(pl.LightningModule):
    def __init__(self, learning_rate=1e-3):
        super(FashionMNISTLitModel, self).__init__()
        self.model = LightweightFashionMNIST()
        self.learning_rate = learning_rate

        # Метрики для обучения и валидации
        self.train_accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=10)
        self.val_accuracy = torchmetrics.Accuracy(task='multiclass',num_classes=10)
        self.val_precision = torchmetrics.Precision(task='multiclass',num_classes=10, average='macro')
        self.val_recall = torchmetrics.Recall(task='multiclass', num_classes=10, average='macro')
        self.val_auroc = torchmetrics.AUROC(task='multiclass',num_classes=10)
        self.confmat = torchmetrics.ConfusionMatrix(task='multiclass', num_classes=10)
        self.val_f1 = MulticlassF1Score( num_classes=10, average='macro')  # Добавлена метрика F1


    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.train_accuracy.update(preds, y)
        self.log('train/loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    # Вместо training_epoch_end, используем on_train_epoch_end (без аргументов)
    def on_train_epoch_end(self):
        epoch_time = time.time() - self.epoch_start_time
        self.log('train/epoch_time', epoch_time)
        acc = self.train_accuracy.compute()
        self.log('train/accuracy', acc, prog_bar=True)
        self.train_accuracy.reset()
        self.log('train/lr', self.learning_rate)

    def on_train_epoch_start(self):
        self.epoch_start_time = time.time()

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.val_accuracy.update(preds, y)
        self.val_precision.update(preds, y)
        self.val_recall.update(preds, y)
        self.val_auroc.update(F.softmax(logits, dim=1), y)
        self.val_f1.update(preds, y)  # Обновление F1 метрики

        self.confmat.update(preds, y)
        self.log('val/loss', loss, prog_bar=True, on_epoch=True)
        return loss

    def on_validation_epoch_end(self):
        acc = self.val_accuracy.compute()
        prec = self.val_precision.compute()
        rec = self.val_recall.compute()
        f1 = self.val_f1.compute()  # Вычисление F1 метрики
        auroc = self.val_auroc.compute()
        self.log('val/accuracy', acc, prog_bar=True)
        self.log('val/precision', prec)
        self.log('val/recall', rec)
        self.log('val/auroc', auroc)
        self.log('val/f1', f1)  # Логирование F1 метрики

        # Логируем матрицу ошибок как изображение в TensorBoard
        confmat = self.confmat.compute().cpu().numpy()
        fig = self.plot_confusion_matrix(confmat)
        self.logger.experiment.add_figure("Confusion Matrix", fig, self.current_epoch)

        self.val_accuracy.reset()
        self.val_precision.reset()
        self.val_recall.reset()
        self.val_auroc.reset()
        self.val_f1.reset()  # Сброс F1 метрики
        self.confmat.reset()

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def plot_confusion_matrix(self, confmat):
        fig, ax = plt.subplots(figsize=(8, 8))
        im = ax.imshow(confmat, interpolation='nearest', cmap=plt.cm.Blues)
        ax.figure.colorbar(im, ax=ax)
        ax.set(xticks=np.arange(confmat.shape[1]),
               yticks=np.arange(confmat.shape[0]),
               xticklabels=np.arange(10),
               yticklabels=np.arange(10),
               ylabel='True label',
               xlabel='Predicted label',
               title='Confusion Matrix')
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
        thresh = confmat.max() / 2.
        for i in range(confmat.shape[0]):
            for j in range(confmat.shape[1]):
                ax.text(j, i, format(confmat[i, j], 'd'),
                        ha="center", va="center",
                        color="white" if confmat[i, j] > thresh else "black")
        fig.tight_layout()
        return fig

# Подготовка данных
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = FashionMNIST(root=dataset_path, train=True, download=True, transform=transform)
val_dataset   = FashionMNIST(root=dataset_path, train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)





In [ ]:

# Инициализируем модель
learn_id = str(uuid.uuid4())
model_name = 'fashion_MNIST_lite'
model = FashionMNISTLitModel(learning_rate=1e-3)
# Настраиваем TensorBoard логгер
logger = TensorBoardLogger(
        logs_path,
        name=model_name,
        version=learn_id,
        sub_dir=(_sd:=f"{(u_:=datetime.utcnow().strftime('%Y_%m_%d__%H_%M_%S'))}"),
)
# Настраиваем колбэк для сохранения модели
checkpoint_callback = ModelCheckpoint(
    monitor=(_tg_m:='val/auroc'),       # Метрика для мониторинга
    dirpath=test_weights_path, # Каталог для сохранения моделей
    filename=f'{model_name}_{_sd}_{learn_id}' + '-{epoch:02d}-{' + _tg_m + ':.2f}', # Имя файла
    save_top_k=1,             # Сохранять только лучшую модель
    mode='max'                # Минимизировать monitored метрику
)

# Создаем тренер PyTorch Lightning
trainer = pl.Trainer(
    max_epochs=20,
    logger=logger,
    log_every_n_steps=50,
    callbacks=[checkpoint_callback],
)
# Запускаем обучение
trainer.fit(model, train_loader, val_loader)

In [18]:
best_model_path = checkpoint_callback.best_model_path

target_path_to_model = os.path.join(prod_weights_path, os.path.split(os.path.split(best_model_path)[0])[1] + '_' + os.path.split(best_model_path)[1])

shutil.move(best_model_path, target_path_to_model)


FileNotFoundError: [Errno 2] No such file or directory: '/workspace/NN/my_checkpoints/fashion_MNIST_lite_2025_02_27__11_35_34_af5d8943-4cf8-4204-b7a3-f39e5a49e673-epoch=10-val/auroc=0.99.ckpt'

In [8]:
target_path_to_model = '/workspace/NN/neural/weights/prod/fashion_MNIST_lite_2025_02_27__11_35_34_af5d8943-4cf8-4204-b7a3-f39e5a49e673-epoch=10-val_auroc=0.99.ckpt'
trained_model = FashionMNISTLitModel.load_from_checkpoint(
    checkpoint_path=target_path_to_model,
    map_location=torch.device(device=device)  # или 'cuda' для GPU
)
model_name='fashion_MNIST_lite'
learn_id = 'af5d8943-4cf8-4204-b7a3-f39e5a49e673'
torch.save(trained_model.model.state_dict(), os.path.join(os.path.split(target_path_to_model)[0], f'{model_name}_{learn_id}.pth'))

In [11]:
from src.configs.config import CONFIG
from huggingface_hub import hf_hub_download
import os
import shutil

if True or os.path.isfile(CONFIG.MODEL_DIR):
    model_path = hf_hub_download(repo_id=CONFIG.AI_WEIGHTS_REPO, filename=CONFIG.AI_WEIGHTS_REPO_FILENAME, force_download=True, local_dir= os.path.join(ROOT_DIR, 'neural', 'weights',  'prod'))
    source_dir = model_path
    target_dir = CONFIG.MODEL_DIR
    # shutil.move(source_dir, target_dir)

In [8]:
!apt-get update
!apt-get install -y libsasl2-dev python3-dev gcc g++
!pip install sasl
!pip install happybase thrift_sasl sasl thrift


Hit:1 http://deb.debian.org/debian bullseye InRelease
Hit:3 http://deb.debian.org/debian-security bullseye-security InRelease        
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/debian11/x86_64  InRelease
Hit:4 http://deb.debian.org/debian bullseye-updates InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsasl2-dev is already the newest version (2.1.27+dfsg-2.1+deb11u1).
g++ is already the newest version (4:10.2.1-1).
gcc is already the newest version (4:10.2.1-1).
python3-dev is already the newest version (3.9.2-3).
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.
  Using cached sasl-0.3.1.tar.gz (44 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [26 lines of output]
      running bdist_wheel
      running build
      running build_py
      crea

In [1]:
! apt-get update
! apt-get install libkrb5-dev -y
!pip install requests requests-kerberos


Hit:2 http://deb.debian.org/debian bullseye InRelease
Get:3 http://deb.debian.org/debian-security bullseye-security InRelease [27.2 kB]
Get:4 http://deb.debian.org/debian bullseye-updates InRelease [44.1 kB]
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/debian11/x86_64  InRelease
Get:5 http://deb.debian.org/debian-security bullseye-security/main amd64 Packages [350 kB]
Fetched 421 kB in 1s (309 kB/s)   
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  comerr-dev krb5-multidev libcom-err2 libgssapi-krb5-2 libgssrpc4
  libk5crypto3 libkadm5clnt-mit12 libkadm5srv-mit12 libkdb5-10 libkrb5-3
  libkrb5support0
Suggested packages:
  doc-base krb5-doc krb5-user
Recommended packages:
  krb5-locales
The following NEW packages will be installed:
  comerr-dev krb5-multidev libgssrpc4 libkadm5clnt-mit12 libkadm5srv-mit12
  libkdb5-10 libkrb5-dev
The f

In [10]:
import base64
import requests
from requests_kerberos import HTTPKerberosAuth, OPTIONAL

# Настройка Kerberos‑аутентификации для HTTP
kerberos_auth = HTTPKerberosAuth(mutual_authentication=OPTIONAL)

# URL HBase REST сервера
BASE_URL = "http://hbase-rest:8080"

def b64(s: str) -> str:
    """Преобразует строку в base64."""
    return base64.b64encode(s.encode()).decode()

def check_table_exists(table_name: str) -> bool:
    url = f"{BASE_URL}/{table_name}/schema"
    response = requests.get(url, auth=kerberos_auth)
    if response.status_code == 200:
        print(f"Таблица {table_name} существует.")
        return True
    elif response.status_code == 404:
        print(f"Таблица {table_name} не найдена.")
        return False
    else:
        print(f"Ошибка проверки таблицы {table_name}: {response.status_code}")
        return False

def create_table(table_name: str):
    # XML-схема таблицы. Здесь создается одна колонка "cf".
    schema_xml = f"""<TableSchema name="{table_name}">
  <ColumnSchema name="cf" />
</TableSchema>"""
    url = f"{BASE_URL}/{table_name}/schema"
    headers = {'Content-Type': 'text/xml'}
    response = requests.post(url, data=schema_xml, headers=headers, auth=kerberos_auth)
    if response.status_code == 201:
        print(f"Таблица {table_name} успешно создана.")
    else:
        print(f"Не удалось создать таблицу {table_name}. Статус: {response.status_code}. Ответ: {response.text}")

def insert_row(table_name: str, row_key: str, data: dict):
    """
    Вставка строки в таблицу через HBase REST API.
    Формируем XML, где ключ и значения кодируются в base64.
    """
    cells = ""
    for col, val in data.items():
        column = b64("cf:" + col)
        value = b64(val)
        cells += f'<Cell column="{column}">{value}</Cell>\n'
    cellset_xml = f"""<CellSet>
  <Row key="{b64(row_key)}">
    {cells.strip()}
  </Row>
</CellSet>"""
    url = f"{BASE_URL}/{table_name}/{row_key}"
    headers = {'Content-Type': 'text/xml'}
    response = requests.put(url, data=cellset_xml, headers=headers, auth=kerberos_auth)
    if response.status_code in [200, 201]:
        print(f"Строка {row_key} вставлена в таблицу {table_name}.")
    else:
        print(f"Ошибка вставки строки {row_key}: {response.status_code}. Ответ: {response.text}")

def get_row(table_name: str, row_key: str):
    url = f"{BASE_URL}/{table_name}/{row_key}"
    headers = {'Accept': 'text/xml'}
    response = requests.get(url, headers=headers, auth=kerberos_auth)
    if response.status_code == 200:
        print(f"Данные строки {row_key}:")
        print(response.text)
    else:
        print(f"Ошибка получения строки {row_key}: {response.status_code}. Ответ: {response.text}")

def delete_row(table_name: str, row_key: str):
    url = f"{BASE_URL}/{table_name}/{row_key}"
    response = requests.delete(url, auth=kerberos_auth)
    if response.status_code == 200:
        print(f"Строка {row_key} удалена из таблицы {table_name}.")
    else:
        print(f"Ошибка удаления строки {row_key}: {response.status_code}. Ответ: {response.text}")

def main():
    table1 = "table1"
    table2 = "table2"

    # Создание таблиц, если они отсутствуют
    if not check_table_exists(table1):
        create_table(table1)
    if not check_table_exists(table2):
        create_table(table2)

    # Пример CRUD операций на таблице table1
    row_key = "row1"
    data = {"col1": "value1", "col2": "value2"}

    print("\nВставка строки:")
    insert_row(table1, row_key, data)

    print("\nПолучение строки:")
    get_row(table1, row_key)

    print("\nУдаление строки:")
    delete_row(table1, row_key)

if __name__ == "__main__":
    main()


Таблица table1 не найдена.
Таблица table1 успешно создана.
Таблица table2 не найдена.
Таблица table2 успешно создана.

Вставка строки:
Строка row1 вставлена в таблицу table1.

Получение строки:
Данные строки row1:
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><CellSet><Row key="cm93MQ=="><Cell column="Y2Y6Y29sMQ==" timestamp="1741605695067">dmFsdWUx</Cell><Cell column="Y2Y6Y29sMg==" timestamp="1741605695067">dmFsdWUy</Cell></Row></CellSet>

Удаление строки:
Строка row1 удалена из таблицы table1.


In [11]:
!apt-get install curl -y
!curl http://hbase-rest:8080/


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
curl is already the newest version (7.74.0-1.3+deb11u14).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
table1
table2


In [13]:
import base64
import requests

def encode_str(s):
    """Кодирует строку в Base64."""
    return base64.b64encode(s.encode("utf-8")).decode("utf-8")

def check_table_exists(base_url, table):
    url = f"{base_url}/{table}/schema"
    headers = {"Accept": "application/xml"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        print(f"Таблица '{table}' существует.")
        return True
    elif response.status_code == 404:
        print(f"Таблица '{table}' не найдена.")
        return False
    else:
        print(f"Ошибка при проверке таблицы '{table}': {response.status_code}")
        return False

def create_table(base_url, table):
    url = f"{base_url}/{table}/schema"
    schema_xml = f"""<?xml version="1.0"?>
<TableSchema name="{table}">
  <ColumnSchema name="cf"/>
</TableSchema>"""
    headers = {"Content-Type": "text/xml"}
    response = requests.post(url, data=schema_xml, headers=headers)
    if response.status_code in (200, 201):
        print(f"Таблица '{table}' успешно создана.")
    else:
        print(f"Не удалось создать таблицу '{table}': {response.status_code}, {response.text}")

def insert_row(base_url, table, row_key, data):
    """
    Вставляет строку в таблицу.
    data – словарь вида { "column": "value", ... }.
    В этом примере колонка формируется как 'cf:column'.
    """
    row_encoded = encode_str(row_key)
    cells = ""
    for col, val in data.items():
        col_full = f"cf:{col}"
        cells += f'<Cell column="{encode_str(col_full)}">{encode_str(val)}</Cell>'
    cellset_xml = f"""<?xml version="1.0"?>
<CellSet>
  <Row key="{row_encoded}">
    {cells}
  </Row>
</CellSet>"""
    url = f"{base_url}/{table}/{row_key}"
    headers = {"Content-Type": "text/xml"}
    response = requests.put(url, data=cellset_xml, headers=headers)
    if response.status_code in (200, 201):
        print(f"Строка '{row_key}' вставлена в таблицу '{table}'.")
    else:
        print(f"Ошибка вставки строки '{row_key}' в таблицу '{table}': {response.status_code}, {response.text}")

def get_row(base_url, table, row_key):
    url = f"{base_url}/{table}/{row_key}"
    headers = {"Accept": "text/xml"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        print(f"Строка '{row_key}' получена:")
        print(response.text)
        return response.text
    else:
        print(f"Ошибка получения строки '{row_key}' из таблицы '{table}': {response.status_code}, {response.text}")
        return None

def update_row(base_url, table, row_key, data):
    # Обновление строки производится путем повторного вызова insert_row,
    # так как HBase REST перезаписывает данные для указанного row_key.
    print(f"Обновление строки '{row_key}' в таблице '{table}'...")
    insert_row(base_url, table, row_key, data)

def delete_row(base_url, table, row_key):
    url = f"{base_url}/{table}/{row_key}"
    response = requests.delete(url)
    if response.status_code == 200:
        print(f"Строка '{row_key}' удалена из таблицы '{table}'.")
    else:
        print(f"Ошибка удаления строки '{row_key}' из таблицы '{table}': {response.status_code}, {response.text}")

def main():
    base_url = "http://hbase-rest:8080"  # Адрес REST-сервера HBase
    table1 = "table1"
    table2 = "table2"

    # Проверяем наличие таблиц, создаем, если отсутствуют
    if not check_table_exists(base_url, table1):
        create_table(base_url, table1)
    if not check_table_exists(base_url, table2):
        create_table(base_url, table2)

    # Пример операций для таблицы table1
    row_key = "row1"
    initial_data = {"col1": "value1", "col2": "value2"}
    updated_data = {"col1": "new_value1", "col3": "value3"}

    # Вставка строки
    insert_row(base_url, table1, row_key, initial_data)

    # Получение строки
    get_row(base_url, table1, row_key)

    # Обновление строки
    update_row(base_url, table1, row_key, updated_data)

    # Получение обновленной строки
    get_row(base_url, table1, row_key)

    # Удаление строки
    delete_row(base_url, table1, row_key)

    # Попытка получения удаленной строки
    get_row(base_url, table1, row_key)

if __name__ == "__main__":
    main()


Ошибка при проверке таблицы 'table1': 406
Таблица 'table1' успешно создана.
Ошибка при проверке таблицы 'table2': 406
Таблица 'table2' успешно создана.
Строка 'row1' вставлена в таблицу 'table1'.
Строка 'row1' получена:
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><CellSet><Row key="cm93MQ=="><Cell column="Y2Y6Y29sMQ==" timestamp="1741606102798">dmFsdWUx</Cell><Cell column="Y2Y6Y29sMg==" timestamp="1741606102798">dmFsdWUy</Cell></Row></CellSet>
Обновление строки 'row1' в таблице 'table1'...
Строка 'row1' вставлена в таблицу 'table1'.
Строка 'row1' получена:
<?xml version="1.0" encoding="UTF-8" standalone="yes"?><CellSet><Row key="cm93MQ=="><Cell column="Y2Y6Y29sMQ==" timestamp="1741606102808">bmV3X3ZhbHVlMQ==</Cell><Cell column="Y2Y6Y29sMg==" timestamp="1741606102798">dmFsdWUy</Cell><Cell column="Y2Y6Y29sMw==" timestamp="1741606102808">dmFsdWUz</Cell></Row></CellSet>
Строка 'row1' удалена из таблицы 'table1'.
Ошибка получения строки 'row1' из таблицы 'table1': 404, Not found
